In [3]:
:dep arrow = "4.3.0"
:dep rayon = "1.5.1"

In [4]:
use std::fs::File;
use arrow::csv;
use arrow::array::{Array, Float64Array};
use rayon::prelude::*;

In [5]:
const FRAME_SIZE: usize = 300;
const RATIO: f64 = 11.1;

In [6]:
fn is_fake_max_age(age: &Float64Array, i: usize) -> bool {
    let mut is_max = true;
    for j in (i + 1)..(i + 1 + FRAME_SIZE).min(age.len()) {
	if !age.is_null(j) && !age.is_null(i) &&
	    age.value(j) > age.value(i) * RATIO {
		is_max = false;
		break
	}
    }
    is_max
}

In [16]:
let input_path = "../confirmed-cases.csv";
let input_file = File::open(input_path).unwrap();
let builder = csv::ReaderBuilder::new()
    .infer_schema(Some(134000))
    .has_header(true)
    .with_batch_size(1000000);
let mut reader = builder.build(input_file).unwrap();
let batch = reader.next().unwrap().unwrap();

In [25]:
let age_i = batch.schema().index_of("age").unwrap();
let max_age: Vec<_> = (0..batch.column(age_i).len()).collect::<Vec<usize>>()
    .into_par_iter()
    .map(|i| is_fake_max_age(batch.column(age_i).as_any().downcast_ref::<Float64Array>().unwrap(), i))
    .collect();

In [26]:
println!("{} {}", max_age.len(), max_age[10000])

236060 true


()